In [51]:
#Import necessary dependencies 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests 
import csv 
import datetime



In [52]:
# Per instructions we needed to create a file path for our output data to be stored. What should appear in this file by the end of this code will be a data frame of the information collected
City_data_file = "Resources/City_Data.csv"

In [53]:
# need to import citipy to help tag geo coordinates to cities selected
from citipy import citipy 

In [54]:
# import my individualized API key and define desired units for temperature because I will need to successfuly call url 
from BK_api_key import BK_api_key

temperature_units = "Imperial"


In [55]:
# Prior to any list creation or defining range for specific values we will look for, I need make sure my query_url works. Base url was provided from OpenWeatherMap recommendations on how to perform successful API call 
url_query = f"http://api.openweathermap.org/data/2.5/weather?appid={BK_api_key}&units={temperature_units}&q="

In [56]:
# Since we will be capturing the lat and long of the random cities I will need define ranges for those values and create an open list to add those values later on. Range values are based on researched max and min values for lat and long  

latitude_range = (-90, 90)
longitude_range = (-180, 180)


latit_and_longtit_list = []
cities = []


In [57]:
# Prior to generating for loops to capture my city data, I want to set count value variables that will help organize through the different lists I will be appending information to. 
count_one = 0
set_one = 1


In [58]:
#From starter code
# Need to use np module to generate selection of the random cities and for those to be stored in the open lists I created above. Obviously what's different is how I named my variables.
latitude_values = np.random.uniform(low=-90.000, high=90.000, size=1500)
longitude_values = np.random.uniform(low=-180.000, high=180.000, size=1500)

latit_and_longtit_list = zip(latitude_values, longitude_values)

# For loop that will help identify nearest city (using citipy) for each latitude and longitude combination found
for latit_and_longtit_list in latit_and_longtit_list:
    city = citipy.nearest_city(latit_and_longtit_list[0], latit_and_longtit_list[1]).city_name
    
    # Conditional statement to ensure that only unquie city values are added to our open cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

612

In [59]:
#Print statement to see what the output is for the first row. This will help when organizing my df later
print(cities[0])

beringovskiy


In [60]:
# Prior to running code through my randomly selected cities, I need to create additional open lists that will hold values that will serve as the column values in my data frame that I will create. Since open list for the lats and longs have been created, I wont add them here. 
country_data = []
cities_df_list = []
humidity_data = []
dates_data = []
temperature_max_values = []
wind_speed_data = []